<a href="https://colab.research.google.com/github/Uniholder/Specialization-Yandex-MIPT/blob/main/6_Final_projects/2_churn_prediction/week_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir data
!wget -O data/orange_small_churn_data.train https://stepik.org/media/attachments/lesson/90608/_847b6cd041836e8fa914243e83d898fb_orange_small_churn_data.train
!wget -O data/orange_small_churn_labels.train https://stepik.org/media/attachments/lesson/90608/_ce5e9e4280a8856218f1886f007de2a4_orange_small_churn_labels.train
!gdown -O data/ 1rW8ZBa870U3DHzERRfy1q777914JKPtA

--2022-08-17 11:27:19--  https://stepik.org/media/attachments/lesson/90608/_847b6cd041836e8fa914243e83d898fb_orange_small_churn_data.train
Resolving stepik.org (stepik.org)... 13.69.75.239
Connecting to stepik.org (stepik.org)|13.69.75.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23722578 (23M) [application/octet-stream]
Saving to: ‘data/orange_small_churn_data.train’

data/orange_small_c 100%[===================>]  22.62M  11.8MB/s    in 1.9s    

2022-08-17 11:27:25 (11.8 MB/s) - ‘data/orange_small_churn_data.train’ saved [23722578/23722578]

--2022-08-17 11:27:25--  https://stepik.org/media/attachments/lesson/90608/_ce5e9e4280a8856218f1886f007de2a4_orange_small_churn_labels.train
Resolving stepik.org (stepik.org)... 13.69.75.239
Connecting to stepik.org (stepik.org)|13.69.75.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117024 (114K) [application/octet-stream]
Saving to: ‘data/orange_small_churn_labels.train’

data/ora

# Входные данные

In [ ]:
import pandas as pd
import os
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

PATH_TO_DATA = 'data'

In [ ]:
df_raw = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_data.train'))
labels = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_labels.train'), header=None, squeeze=True)
df_raw.shape

(40000, 230)

In [ ]:
num_vars = df_raw.columns[:190]
cat_vars = df_raw.columns[190:]
na_vars = df_raw.columns[df_raw.isna().all()]

nna_num_vars = list(set(num_vars) - set(na_vars))
nna_cat_vars = list(set(cat_vars) - set(na_vars))

In [ ]:
df_raw['target'] = labels
df = df_raw.dropna(axis=1, how='all')
df.shape

(40000, 213)

Отделим hold-out dataset

In [ ]:
df_train, df_ho = train_test_split(df, test_size=5000)
df_train.shape, df_ho.shape

((35000, 213), (5000, 213))

In [ ]:
# df_ho.to_csv(os.path.join(PATH_TO_DATA, 'hold_out_dataset.csv'))

Preprocessing

In [ ]:
cols = df_train.drop('target', axis=1).columns

In [ ]:
cat_encoder = ColumnTransformer(
    transformers=[
        ("cat", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), nna_cat_vars)
    ], remainder='passthrough'
)

preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('cat_encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

In [ ]:
%%time
df_train_enc = pd.DataFrame(preprocessor.fit_transform(df_train[cols]), columns=cols)

CPU times: user 6.74 s, sys: 751 ms, total: 7.49 s
Wall time: 7.49 s


In [ ]:
df_train_enc

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,0.0,0.0,0.0,0.0,0.0,123.0,1.0,0.0,0.0,0.0,...,2652.0,4.0,1004.0,0.0,0.0,1.0,6.0,2.0,8.0,0.0
1,0.0,0.0,0.0,0.0,0.0,271.0,4.0,0.0,0.0,0.0,...,305.0,4.0,2352.0,3.0,0.0,1.0,18.0,3.0,15.0,0.0
2,0.0,0.0,0.0,0.0,0.0,65.0,0.0,0.0,0.0,0.0,...,1465.0,4.0,1889.0,0.0,0.0,2.0,10.0,2.0,8.0,1.0
3,0.0,0.0,0.0,0.0,60.0,0.0,1.0,0.0,0.0,0.0,...,3637.0,4.0,3572.0,0.0,0.0,0.0,0.0,2.0,8.0,0.0
4,0.0,0.0,0.0,0.0,0.0,111.0,1.0,0.0,0.0,0.0,...,2285.0,4.0,1421.0,3.0,0.0,0.0,10.0,2.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34995,0.0,0.0,0.0,0.0,0.0,126.0,1.0,0.0,0.0,0.0,...,260.0,4.0,1610.0,1.0,0.0,1.0,0.0,2.0,2.0,0.0
34996,0.0,0.0,0.0,0.0,0.0,122.0,1.0,0.0,0.0,0.0,...,3010.0,4.0,3546.0,0.0,0.0,0.0,10.0,2.0,8.0,0.0
34997,0.0,0.0,0.0,0.0,0.0,95.0,1.0,0.0,0.0,0.0,...,1807.0,6.0,566.0,0.0,0.0,2.0,13.0,1.0,29.0,0.0
34998,0.0,0.0,0.0,0.0,0.0,124.0,1.0,0.0,0.0,0.0,...,71.0,4.0,734.0,0.0,0.0,0.0,12.0,2.0,8.0,0.0


In [ ]:
df_train_enc['Var229'].value_counts()

0.0    28081
1.0     6866
3.0       27
2.0       26
Name: Var229, dtype: int64

In [ ]:
%%time
df_ho_enc = pd.DataFrame(preprocessor.transform(df_ho[cols]), columns=cols)

CPU times: user 3.04 s, sys: 38.1 ms, total: 3.08 s
Wall time: 3.27 s


# Модель

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit, cross_validate
from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, roc_auc_score

from catboost import CatBoostClassifier, cv, Pool

In [ ]:
X, y = df_train_enc, df_train.target
X_val, y_val = df_ho_enc, df_ho.target
cv_scheme = StratifiedShuffleSplit(random_state=0, )

## CatBoost out of the box

In [ ]:
df_train[cols]

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
8988,NaN,NaN,NaN,NaN,NaN,861.0,7.0,NaN,NaN,NaN,...,gmDv8BP,oslk,HCTwS9i,LM8l689qOp,NaN,kG3k,Aoh3,RAYp,F2FyR07IdsN7I,am7c
12847,NaN,NaN,NaN,NaN,NaN,1897.0,28.0,NaN,NaN,NaN,...,4UxGlow,oslk,catzS2D,jySVZNlOJy,NaN,kG3k,szEZ,ZI9m,VjDE,am7c
28412,NaN,NaN,NaN,NaN,NaN,455.0,0.0,NaN,NaN,NaN,...,OXxv4zN,oslk,VWtE4Qn,LM8l689qOp,NaN,xG3x,Qu4f,RAYp,F2FyR07IdsN7I,mj86
27589,NaN,NaN,NaN,NaN,31485.0,NaN,NaN,NaN,0.0,NaN,...,zLzYH7D,oslk,xb5Ty9x,LM8l689qOp,NaN,NaN,3Cy4,RAYp,F2FyR07IdsN7I,NaN
12556,NaN,NaN,NaN,NaN,NaN,777.0,7.0,NaN,NaN,NaN,...,aD1vOIl,oslk,NnYzgmc,jySVZNlOJy,NaN,NaN,Qu4f,RAYp,55YFVY9,am7c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35470,NaN,NaN,NaN,NaN,NaN,882.0,7.0,NaN,NaN,NaN,...,3s4OZ4E,oslk,QpZO8fW,M_8D,NaN,kG3k,3Cy4,RAYp,55YFVY9,am7c
39476,NaN,NaN,NaN,NaN,NaN,854.0,7.0,NaN,NaN,NaN,...,n2dvEWW,oslk,xInqI2t,LM8l689qOp,NaN,NaN,Qu4f,RAYp,F2FyR07IdsN7I,NaN
29123,NaN,NaN,NaN,NaN,NaN,665.0,7.0,NaN,NaN,NaN,...,UF16siJ,zCkv,APgdzOv,LM8l689qOp,NaN,xG3x,Xa3G,6fzt,xwM2aC7IdeMC0,am7c
8642,NaN,NaN,NaN,NaN,NaN,868.0,NaN,NaN,NaN,NaN,...,1FARoKG,oslk,CnZoIp0,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN


In [ ]:
si = SimpleImputer(strategy='most_frequent')
X_imp = pd.DataFrame(si.fit_transform(df_train[cols]), columns=cols)
X_val_imp = pd.DataFrame(si.transform(df_ho[cols]), columns=cols)

In [ ]:
base_params = {
    'eval_metric': 'AUC:hints=skip_train~false', 
    'verbose': False,
    'loss_function': 'Logloss'
}

In [ ]:
%%time
cat_boost_cl_oob = CatBoostClassifier(**base_params, cat_features=nna_cat_vars)
cat_boost_cl_oob.fit(X_imp, y)
print(roc_auc_score(y_val, cat_boost_cl_oob.predict_proba(X_val_imp)[:, 1]))
print(classification_report(y_val, cat_boost_cl_oob.predict(X_val_imp)))

0.7556119599920575
              precision    recall  f1-score   support

          -1       0.93      1.00      0.96      4629
           1       0.40      0.01      0.02       371

    accuracy                           0.93      5000
   macro avg       0.66      0.50      0.49      5000
weighted avg       0.89      0.93      0.89      5000

CPU times: user 8min 49s, sys: 29 s, total: 9min 18s
Wall time: 5min 3s


In [ ]:
%%time
cv_results = cv(
    params={**base_params},
    pool=Pool(X_imp, y, cat_features=nna_cat_vars),
    fold_count=5,
    shuffle=True,
    # plot=True,
    type='Classical'
)

Training on fold [0/5]

bestTest = 0.736551783
bestIteration = 673

Training on fold [1/5]

bestTest = 0.7489355096
bestIteration = 857

Training on fold [2/5]

bestTest = 0.7328463226
bestIteration = 701

Training on fold [3/5]

bestTest = 0.7424870962
bestIteration = 871

Training on fold [4/5]

bestTest = 0.7397731753
bestIteration = 497

CPU times: user 40min 24s, sys: 3min 24s, total: 43min 48s
Wall time: 22min 54s


# Тест

In [ ]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(predicted_labels.shape[0]),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
df_test = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_test_data.csv'), index_col=0)
df_test

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
ID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,1225.0,7.0,NaN,NaN,NaN,...,zCkv,APgdzOv,jySVZNlOJy,NaN,ELof,xb3V,6fzt,Zy3gnGM,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,896.0,14.0,NaN,NaN,NaN,...,oslk,IIvC99a,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,791.0,7.0,NaN,NaN,NaN,...,oslk,6YSocsg,LM8l689qOp,NaN,kG3k,rgKb,RAYp,F2FyR07IdsN7I,mj86,NaN
3,NaN,NaN,NaN,NaN,NaN,2296.0,7.0,NaN,NaN,NaN,...,oslk,5nQ7A2G,jySVZNlOJy,NaN,kG3k,rgKb,RAYp,F2FyR07IdsN7I,am7c,NaN
4,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,...,oslk,MI8s5nE,LM8l689qOp,NaN,NaN,7P5s,RAYp,F2FyR07IdsN7I,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,NaN,NaN,NaN,NaN,714.0,7.0,NaN,NaN,NaN,...,zCkv,P6pu4Vl,LM8l689qOp,NaN,ELof,xb3V,ZI9m,R4y5gQQWY8OodqDV,am7c,NaN
9996,NaN,NaN,NaN,NaN,NaN,812.0,7.0,NaN,NaN,NaN,...,oslk,sXbT3Cb,LM8l689qOp,NaN,ELof,WqMG,RAYp,55YFVY9,mj86,NaN
9997,NaN,NaN,NaN,NaN,NaN,819.0,7.0,NaN,NaN,NaN,...,oslk,05jAV0N,M_8D,NaN,ELof,TNEC,RAYp,55YFVY9,am7c,NaN


In [ ]:
df_test.drop(na_vars, axis=1, inplace=True)
df_test_enc = pd.DataFrame(preprocessor.transform(df_test), columns=cols)
df_test_imp = pd.DataFrame(si.transform(df_test[cols]), columns=cols)

In [ ]:
submission = cat_boost_cl_oob.predict_proba(df_test_imp)[:, 1]

In [ ]:
write_to_submission_file(submission, 'submission.csv', target='result', index_label='ID')

0.71565